<a href="https://colab.research.google.com/github/ryeorin/Colab/blob/main/LSTM_parctice_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Torchtext

In [ ]:
!pip install torchtext==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.8 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.16.0
    Uninstalling torchtext-0.16.0:
      Successfully uninstalled torchtext-0.16.0


In [ ]:
from torchtext import data,datasets
import torch.nn as nn
import torch

In [ ]:
TEXT=data.Field(sequential=True,batch_first=True,lower=True)
LABEL=data.Field(sequential=False,batch_first=True)


# 입력되는 데이터를 정형화 하기 위해
# sequential : 시퀀스 데이터 여부
# use_vocab: 단어 집합을 만들 것인가
# lower: 영어 데이터를 전부 소문자화
# batch_first: 미니 배치 차원을 맨 앞으로 할 것인가
# fix_length: 최대 허용 길이. 이 길이에 맞춰서 패딩

In [ ]:
trainset, testset=datasets.IMDB.splits(TEXT,LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:18<00:00, 4.62MB/s]


In [ ]:
print("학습 데이터 크기: ", len(trainset))

학습 데이터 크기:  25000


In [ ]:
print(vars(trainset[0])) #딕셔너리 형태로 분리되어 있음

{'text': ['well,', "i'll", 'be', 'honest:', 'it', 'is', 'not', 'exactly', 'a', 'sholay.', 'but', 'you', 'cant', 'get', 'a', 'sholay', 'every', 'week.', 'in', 'fact,', 'you', 'could', 'see', 'distinct', 'signatures', 'of', '"not', 'without', 'my', 'daughter"(sally', 'field,', '1991)', 'in', 'this', 'movie.', 'however,', 'as', 'most', '"inspired"', 'movies', 'go,', 'this', 'one', 'was', 'a', 'well-inspired', 'one,', 'well', 'handled', 'and', 'well', 'done.', 'nana', 'patekar,', 'as', 'usual,', 'tends', 'to', 'overdo', 'his', 'hysterics,', 'but', 'all', 'others', 'are', 'commendable.', 'specially', 'so', 'about', 'dipti', 'naval:', 'saw', 'her', 'after', 'a', 'long', 'time,', 'but', 'she', "hasn't", 'lost', 'any', 'of', 'her', 'grace.', 'in', 'fact,', 'she', 'has', 'performed', 'much', 'better', 'that', 'when', 'i', 'last', 'saw', 'her.', 'another', 'one', 'of', 'the', 'bollywood', 'stars', 'that', 'seem', 'to', 'grow', 'more', 'beautiful', 'as', 'they', 'age?<br', '/><br', '/>all', 'in',

In [ ]:
print(trainset.fields.items())

dict_items([('text', <torchtext.data.field.Field object at 0x7b5e475bdba0>), ('label', <torchtext.data.field.Field object at 0x7b5e475be320>)])


In [ ]:
TEXT.build_vocab(trainset,min_freq=10,max_size=10000) # 10번 이상 나온 단어만 단어 집합으로 쓰겠다.

print("단어 집합 크기:", len(TEXT.vocab))

단어 집합 크기: 10002


In [ ]:
print(TEXT.vocab.stoi) # 각 단어별로 인덱스를 부여해줌 <unknown> <> special 토큰

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7b5e1c0e5c90>>, {'<unk>': 0, '<pad>': 1, 'the': 2, 'a': 3, 'and': 4, 'of': 5, 'to': 6, 'is': 7, 'in': 8, 'i': 9, 'this': 10, 'that': 11, 'it': 12, '/><br': 13, 'was': 14, 'as': 15, 'for': 16, 'with': 17, 'but': 18, 'on': 19, 'movie': 20, 'his': 21, 'are': 22, 'not': 23, 'film': 24, 'you': 25, 'have': 26, 'he': 27, 'be': 28, 'at': 29, 'one': 30, 'by': 31, 'an': 32, 'they': 33, 'from': 34, 'all': 35, 'who': 36, 'like': 37, 'so': 38, 'just': 39, 'or': 40, 'has': 41, 'her': 42, 'about': 43, "it's": 44, 'some': 45, 'if': 46, 'out': 47, 'what': 48, 'very': 49, 'when': 50, 'more': 51, 'there': 52, 'she': 53, 'would': 54, 'even': 55, 'good': 56, 'my': 57, 'only': 58, 'their': 59, 'no': 60, 'really': 61, 'had': 62, 'which': 63, 'can': 64, 'up': 65, 'were': 66, 'see': 67, 'than': 68, 'we': 69, '-': 70, 'been': 71, 'into': 72, 'get': 73, 'will': 74, 'story': 75, 'much': 76, 'because': 77, 'most': 78, 'how': 7

# 학습 준비

In [ ]:
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchtext import data,datasets

In [ ]:
SEED =5
random.seed(SEED)
torch.manual_seed(SEED)

In [ ]:
TEXT=data.Field(sequential=True,batch_first=True,lower=True)
LABEL=data.Field(sequential=False,batch_first=True)

In [ ]:
trainset, testset=datasets.IMDB.splits(TEXT,LABEL)

In [ ]:
TEXT.build_vocab(trainset, min_freq=10)
LABEL.build_vocab(trainset)

In [ ]:
vocab_size=len(TEXT.vocab)
n_classes=2

In [ ]:
print("단어 집합 크기:", len(TEXT.vocab))
print("클래스의 개수:",n_classes)

단어 집합 크기: 27299
클래스의 개수: 2


In [ ]:
print(TEXT.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7b5de7660850>>, {'<unk>': 0, '<pad>': 1, 'the': 2, 'a': 3, 'and': 4, 'of': 5, 'to': 6, 'is': 7, 'in': 8, 'i': 9, 'this': 10, 'that': 11, 'it': 12, '/><br': 13, 'was': 14, 'as': 15, 'for': 16, 'with': 17, 'but': 18, 'on': 19, 'movie': 20, 'his': 21, 'are': 22, 'not': 23, 'film': 24, 'you': 25, 'have': 26, 'he': 27, 'be': 28, 'at': 29, 'one': 30, 'by': 31, 'an': 32, 'they': 33, 'from': 34, 'all': 35, 'who': 36, 'like': 37, 'so': 38, 'just': 39, 'or': 40, 'has': 41, 'her': 42, 'about': 43, "it's": 44, 'some': 45, 'if': 46, 'out': 47, 'what': 48, 'very': 49, 'when': 50, 'more': 51, 'there': 52, 'she': 53, 'would': 54, 'even': 55, 'good': 56, 'my': 57, 'only': 58, 'their': 59, 'no': 60, 'really': 61, 'had': 62, 'which': 63, 'can': 64, 'up': 65, 'were': 66, 'see': 67, 'than': 68, 'we': 69, '-': 70, 'been': 71, 'into': 72, 'get': 73, 'will': 74, 'story': 75, 'much': 76, 'because': 77, 'most': 78, 'how': 7

In [ ]:
# training set -> split (train 80%, validation 20%)

trainset, validset=trainset.split(split_ratio=0.8)

print("train 데이터의 크기:",len(trainset))
print("valid 데이터의 크기:",len(validset))

train 데이터의 크기: 20000
valid 데이터의 크기: 5000


In [ ]:
train_iter,val_iter,test_iter=data.BucketIterator.splits(
    (trainset, validset,testset), batch_size=64,shuffle=True,repeat=False
)

In [ ]:
# shuffle 해주면 mini_batch 안에 들어가게 된다.

In [ ]:
mini_batch=next(iter(train_iter)) #train 안에 어떤걸 ??가지고 있는지??

print(mini_batch)


[torchtext.data.batch.Batch of size 64]
	[.text]:[torch.LongTensor of size 64x1475]
	[.label]:[torch.LongTensor of size 64]


In [ ]:
# print(mini_batch ,text.shape)
print(mini_batch.text)

tensor([[   10,     7,     3,  ...,     1,     1,     1],
        [21067,     0,     0,  ...,     1,     1,     1],
        [   52,     7,   371,  ...,     1,     1,     1],
        ...,
        [    3,  1056,   431,  ...,     1,     1,     1],
        [    9,   200,    10,  ...,     1,     1,     1],
        [ 1221,  5824,    64,  ...,     1,     1,     1]])


In [ ]:
print("[학습데이터]: %d [검증데이터]: %d [테스트데이터]: %d [단어수]: %d [클래스]: %d " % (

))

TypeError: ignored

In [ ]:
# 하이퍼파라미터 정의
batch_size=64
learning_rate=1e-3
n_epochs=10

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)
print(torch.cuda.get_device_name(0))

cuda:0
Tesla T4


In [ ]:
# 모델 정의
# 모델 객체화 한 것임? 드롭아웃까지
# 그 아래가 모델이 뭐라고?
# logit으로 확률값이 나오게 됨

class Model(nn.Module):
  def __init__(self,n_layers,hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
    self.n_layers=n_layers
    self.hidden_dim=hidden_dim

    super(Model,self).__init__()

    self.embed=nn.Embeding(n_vocab,embed_dim) # 임베딩으로 변경
    self.lstm=nn.LSTM(embed_dim,self.hidden_dim,num_layers=self.n_layers,batch_first=True)
    self.out=nn.Linear(self.hidden_dim,n_classes)
    self.dropout=nn.Dropout(dropout_p)

  def forward(self, x):
    x=self.embed(x)

    output,(hn,cn)=self.lstm(x)
    hn=hn[1]
    hn=self.dropout(hn)
    logit=self.out(hn)

    return logit

In [ ]:
# train, evaluate 함수 정의
def train(model, optimizer, train_iter):
  model.train()# 모드 변환 중요
  for _.mini_batch in enumerate(train_iter):
    x,y=mini_batch.text.to(device), mini_batch.label.to(device)
    y.data.sub_(1) # 0과 1로 변환 1->0, 2->1

    optimizer.zero_grad() #옵티마이저 쓸때 초기화가 안 되어있기 때문에 초기화 시켜줘야함

    logit=model(x)
    loss=F.corss_entropy(logit,y)
    loss.backward()

    optimizer.step()

def evaluate(model, val_iter):
  model.eval()
  correctes, total_loss=0,0
  for mini_batch in val_iter:
    x,y = mini_batch.text.to(device), mini_batch.label.to(device)
    y.data.sub_(1)

    logit=model(x)
    loss=F.corss_entropy(logit,y,reduction="sum")
    total_loss += loss.item()
    corrects +=(logit.max(1)[1].view(y.size()).data==y.data).sum()

  avg_loss= total_loss/len(val_iter.dataset)
  avg_accuracy=100.0*corrects/len(val_iter.dataset)

  return avg_loss, avg_accuracy

In [ ]:
# iterater 마다 미니 배치 넣어줘야되니가 for문 작성해줌
# 이벨류에이트 모델 검증하는 거
# corrects 맞춘 개수만 입력이 가능
# 평균 로스와 정확도

In [ ]:
model=Model(2,256,vocab_size=400,n_classes, 0.5).to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

SyntaxError: ignored

In [ ]:
#Model 안에 layer 개수 2개 hidden dim 256 길이
# 임베딩 레이어 통해서 단어가
# 옵티마이저 안에 모델의 파라미터가 업데이트가 됨

In [ ]:
best_val_loss=None
for e in range(1, n_epochs+1):
  train(model, optimizer, train_iter)
  val_loss, val_accuracy=evaluate(model, val_iter)

  print("Epoch: %d | Val loss: %.2f | Val accuracy: %.2f" % (e, val_loss, val_accuracy))

  if not best_val_loss or val_loss < best_val_loss:
    torch.save(model.state_dict(), './txtclassification.pt')
    best_val_loss=val_loss

# 테스트셋 추론 및 결과 확인

In [ ]:
model.load_state_dict(torch.load('./txtclassification.pt'))
test_loss, text_acc=evaluate(model, test_iter)

print("테스트 오차: %5.2f | 테스트 정확도: %5.2f" % (text_loss, text_acc))